In [1]:
import pandas as pd
import numpy as np
from scipy import fft
from sklearn.svm import SVC
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import normalize

class LightFluxProcessor:
    def __init__(self, fourier=True, normalize=True):
        self.fourier = fourier
        self.normalize = normalize

    def process(self, X, Xd):
        if self.normalize:
            print("Normalizing...")
            X = normalize(X)
            Xd = normalize(Xd)

        if self.fourier:
            print("Applying Fourier...")
            X = np.abs(fft.fft(X))
            Xd = np.abs(fft.fft(Xd))

            X = X[:, :(X.shape[1] // 2)]
            Xd = Xd[:, :(Xd.shape[1] // 2)]

        print("Finished Processing!")
        return X, Xd

def main():
    print("Loading datasets...")
    df_train = pd.read_csv("Data/exoTrain.csv", encoding = "ISO-8859-1")
    df_test = pd.read_csv("Data/exoTest.csv", encoding = "ISO-8859-1")

    X_train = np.array(df_train.drop('LABEL', axis=1))
    Y_train = np.array(df_train.LABEL) - 1
    X_test = np.array(df_test.drop('LABEL', axis=1))
    Y_test = np.array(df_test.LABEL) - 1

    LFP = LightFluxProcessor(fourier=True, normalize=False)
    X_train, X_test = LFP.process(X_train, X_test)

    model = SVC(kernel='linear', gamma='scale', class_weight='balanced', probability=True, max_iter=10000)
    print("Training...")
    model.fit(X_train, Y_train)
    print("Finished training!")
    print('')
    print("Making predictions..")
    train_outputs = model.predict(X_train)
    test_outputs = model.predict(X_test)

    train_prob = model.predict_proba(X_train)[:, 1]
    test_prob = model.predict_proba(X_test)[:, 1]
    print("Finished predictions!")

    ap_train = average_precision_score(Y_train, train_prob)
    ap_test = average_precision_score(Y_test, test_prob)
main()

Loading datasets...
Applying Fourier...
Finished Processing!
Training...


C:\Users\vikas\Desktop\exoplanet\env\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Finished training!

Making predictions..
Finished predictions!


In [7]:


class LightFluxProcessor:
    def __init__(self, fourier=True, normalize=True):
        self.fourier = fourier
        self.normalize = normalize

    def process(self, X):
        if self.normalize:
            print("Normalizing data...")
            X = StandardScaler().fit_transform(X)

        if self.fourier:
            print("Applying Fourier transformation...")
            X = np.abs(np.fft.fft(X, axis=1))[:, :X.shape[1] // 2]

        print("Finished processing data!")
        return X

def load_data(train_path, test_path):
    print("Loading datasets...")
    df_train = pd.read_csv(train_path, encoding="ISO-8859-1")
    df_test = pd.read_csv(test_path, encoding="ISO-8859-1")

    X_train = df_train.drop('LABEL', axis=1)
    y_train = df_train['LABEL'] - 1  # Map labels to 0 and 1
    X_test = df_test.drop('LABEL', axis=1)
    y_test = df_test['LABEL'] - 1

    return X_train, y_train, X_test, y_test

def preprocess_data(X_train, y_train, X_test, y_test, augment_train=True, augment_test=True):
    if augment_train:
        print("Augmenting training data...")
        extra = np.flip(X_train, axis=-1)
        X_train = np.concatenate((X_train, extra), axis=0)
        y_train = np.concatenate((y_train, y_train), axis=0)

    if augment_test:
        print("Augmenting test data...")
        dextra = np.flip(X_test, axis=-1)
        X_test = np.concatenate((X_test, dextra), axis=0)
        y_test = np.concatenate((y_test, y_test), axis=0)

    lfp = LightFluxProcessor()
    X_train = lfp.process(X_train)
    X_test = lfp.process(X_test)

    return X_train, y_train, X_test, y_test

def main():
    trainSetPath = "Data/exoTrain.csv"
    testSetPath = "Data/exoTest.csv"

    X_train, y_train, X_test, y_test = load_data(trainSetPath, testSetPath)
    X_train, y_train, X_test, y_test = preprocess_data(X_train, y_train, X_test, y_test)
    train_and_evaluate_model(X_train, y_train, X_test, y_test)

main()


Loading datasets...
Augmenting training data...


InvalidIndexError: (slice(None, None, None), slice(None, None, -1))